In [1]:
import pandas as pd
from bs4 import BeautifulSoup

import asyncio
import nest_asyncio
import aiohttp

In [2]:
history = {
    'authors': [],
    'titles': [],
    'annotations': [],
    'image_urls': [],
    'book_urls': [],
    'genres': []
}

In [3]:
# сюда транслит название с сайты
genres = {
    # Фэнтези
    'Боевое фэнтези': 'boevoe-fentezi',
    'Историческое фэнтези': 'istoricheskoe-fentezi',
    'Городское фэнтези': 'gorodskoe-fentezi',
    'Приключенческое фэнтези': 'priklyuchencheskoe-fentezi',
    'Юмористическое фэнтези': 'yumoristicheskoe-fentezi',
    'Бытовое фэнтези': 'bytovoe-fentezi',
    'Эпическое фэнтези': 'epicheskoe-fentezi',
    'Магическая академия': 'magicheskaya-akademiya',
    'Азиатское фэнтези': 'aziatskoe-fentezi',
    'Cлавянское фэнтези': 'slavyanskoe-fentezi',
    'Тёмное фэнтези': 'temnoe-fentezi',
    
    # Роман
    'Любовное фэнтези': 'lyubovnoe-fentezi',
    'Любовная фантастика': 'lyubovnaya-fantastika',
    'Короткий роман': 'korotkiy-lyubovnyy-roman',
    'Исторический роман': 'istoricheskiy-lyubovnyy-roman',
    'Современный роман': 'sovremennyy-lyubovnyy-roman',
    'Мистический роман': 'misticheskii-lyubovnyi-roman',
    'Криминальный роман': 'kriminalnii-lyubovnyi-roman',
    'Романы о неверности': 'romani-o-neverosti',
    'Романтическая комедия': 'romanticheskaya-komedia',
    'Служебный роман': 'sluzhebnyj-roman',
    'Остросюжетный любовный роман': 'ostrosugetnyj-lyubovnyj-roman',
    
    # Фантастика
    'Боевая фантастика': 'boevaya-fantastika',
    'Научная фантастика': 'nauchnaya-fantastika',
    'Космическая фантастика': 'kosmicheskaya-fantastika',
    'Альтернативная история': 'alternativnaya-istoriya',
    'Постапокалипсис': 'postapokalipsis',
    'Антиутопия': 'antiutopiya',
    'Киберпанк': 'kiberpank',
    'Юмористическая фантастика': 'yumoristicheskaya-fantastika',
    'РеалРПГ': 'realrpg',
    'ЛитРПГ': 'litrpg',
    'Уся (Wuxia)': 'usya-wuxia',
    'Бояръ-аниме': 'bojar-anime',
    
    # Молодежная проза
    'Молодежная мистика': 'molodejnaya-mistika',
    'Студенческий роман': 'studencheskii-roman',
    'Подростковая проза': 'podrostkovaya-proza'
}


MAIN_GENRE = 'Молодежная проза'
SUB_GENRE = 'Подростковая проза'
NAME = genres[SUB_GENRE] # тут менять на название с сайта
PAGES = 150 # выставить кол-во стр, которые нужно спарсить

In [4]:
def save_book_data(history, books):
    BASE_URL = 'https://litnet.com'
    
    for book in books:
        author = book.find_all('a', class_='author')
        author = ', '.join([a.get_text() for a in author])
        
        title = book.find('h4', class_='book-title').get_text(strip=True)
        annotation = book.find('span', itemprop='description').get_text(strip=True)
        image_url = book.find('img', itemprop='image')['src']
        book_url = BASE_URL + books[0].find('a', class_='readmore')['href']
        
        history['authors'].append(author)
        history['titles'].append(title)
        history['annotations'].append(annotation)
        history['image_urls'].append(image_url)
        history['book_urls'].append(book_url)
        history['genres'].append(f'{MAIN_GENRE}, {SUB_GENRE}')

In [5]:
semaphore = asyncio.Semaphore(20)

async def fetch(session, url):
    async with semaphore:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            books = soup.find_all('div', class_='row book-item')
            save_book_data(history, books)
        
        await asyncio.sleep(0.5)

In [6]:
async def main(url, genre, n_pages=10):
    urls = [url.format(genre, genre, i) for i in range(1, n_pages)]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        await asyncio.gather(*tasks)

In [7]:
url = 'https://litnet.com/ru/top/{}?alias={}&page={}'

nest_asyncio.apply()
await main(url, NAME, PAGES)

In [8]:
books_df = pd.DataFrame({
    'author': history['authors'],
    'title': history['titles'],
    'annotation': history['annotations'],
    'img_url': history['image_urls'],
    'book_url': history['book_urls'],
    'genre': history['genres']
})

In [9]:
books_df

,author,title,annotation,img_url,book_url,genre
0,Дина Ареева,"Игры мажоров. ""Сотый"" лицей","— Подумаешь, Топольский! — слышу как возмущает...",https://rust.litnet.com/uploads/covers/120/171...,https://litnet.com/ru/book/igry-mazhorov-sotyi...,"Молодежная проза, Подростковая проза"
1,Валерия Ангелос,Девочка для Бешеного,"— Ты моя на 365 дней, — говорит он и подступае...",https://rust.litnet.com/uploads/covers/120/163...,https://litnet.com/ru/book/igry-mazhorov-sotyi...,"Молодежная проза, Подростковая проза"
2,"Майя Пивоварова, Александра Пивоварова",Грешник против Дьяволицы,На первый взгляд Оля ─ обыкновенная девушка-по...,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/igry-mazhorov-sotyi...,"Молодежная проза, Подростковая проза"
3,Инна Стужева,"Ты влипла, девочка",❤️ БЕСТСЕЛЛЕР ❤️ Я столкнулась с ним поздно ве...,https://rust.litnet.com/uploads/covers/120/163...,https://litnet.com/ru/book/igry-mazhorov-sotyi...,"Молодежная проза, Подростковая проза"
4,"Эля Пылаева, Янка Рам",Она - новенькая,"Новенькая тянет за кончик своей трости, превра...",https://rust.litnet.com/uploads/covers/120/162...,https://litnet.com/ru/book/igry-mazhorov-sotyi...,"Молодежная проза, Подростковая проза"
...,...,...,...,...,...,...
2195,Ownerfather Sergey Grey,Долли и Эдуард,Долли и Эдуард живут на ферме Брэдфорд. Долли ...,https://rust.litnet.com/uploads/covers/120/167...,https://litnet.com/ru/book/podrugi-morya-b423837,"Молодежная проза, Подростковая проза"
2196,Николай Хрипков (Копа),Шли по улице девчонки,"Повесть для детей и для взрослых, их родителей...",https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/podrugi-morya-b423837,"Молодежная проза, Подростковая проза"
2197,Изабелла Некрасова,Бытовые курсы,"В последнее время, я постоянно сталкиваюсь с о...",https://rust.litnet.com/uploads/covers/120/167...,https://litnet.com/ru/book/podrugi-morya-b423837,"Молодежная проза, Подростковая проза"
2198,Ana_Rite,По льду,КНИГА УЖЕ В ПРОДАЖЕ НА ОЗОН И ВБ!\r\n\r\nТрени...,https://rust.litnet.com/uploads/covers/120/170...,https://litnet.com/ru/book/podrugi-morya-b423837,"Молодежная проза, Подростковая проза"


In [10]:
books_df.to_csv(f'{NAME}.csv', index=False)